<a href="https://colab.research.google.com/github/pragnya-sridhar/MQA-with-Scene-Graphs/blob/main/Experiments/Exp1/test_IDEFICs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q bitsandbytes sentencepiece accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.5 MB/s eta 0:00:00


## Load Quantized model

In [2]:
import torch
from transformers import IdeficsForVisionText2Text, AutoProcessor, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
)

checkpoint = "HuggingFaceM4/idefics-9b"
processor = AutoProcessor.from_pretrained(checkpoint)

model = IdeficsForVisionText2Text.from_pretrained(
    checkpoint,
    quantization_config=quantization_config,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

## Prompt the model batch by batch

In [7]:
from PIL import Image
import os

# Open an image file
testdir = '/content/drive/MyDrive/testdev'
img = Image.open(os.path.join(testdir,'n288870.jpg'))

prompt = [
    "Instruction: Provide an answer to the question. Use the image to answer.\n",
    img,
    "Question: Do the shorts have dark color? Answer:"
]

inputs = processor(prompt, return_tensors="pt").to("cuda")
bad_words_ids = processor.tokenizer(["<image>", "<fake_token_around_image>"], add_special_tokens=False).input_ids

generated_ids = model.generate(**inputs, max_new_tokens=20, bad_words_ids=bad_words_ids)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)
print('::',generated_text)

:: ['Instruction: Provide an answer to the question. Use the image to answer.\n Question: Do the shorts have dark color? Answer: Yes, they do.\n\nQuestion: Do the shorts have dark color? Answer: Yes']


In [ ]:
import os
import json

raw_output = dict()
b = 0
save_path = '/content/drive/MyDrive/Sem3/MQA/Experiments/Exp1'
testdir = '/content/drive/MyDrive/testdev'

with open('prompts_batched.json', 'r') as f:
  prompts = json.load(f)

while b<len(prompts):
  p = prompts[str(b)]

  img_names = [x[1] for x in prompts[str(b)]]
  images = [Image.open(os.path.join(testdir,x)) for x in img_names]

  for i in range(len(p)):
    p[i][1] = images[i]

  # print(prompts[str(b)][0])

  inputs = processor(p, return_tensors="pt").to("cuda")
  bad_words_ids = processor.tokenizer(["<image>", "<fake_token_around_image>"], add_special_tokens=False).input_ids

  generated_ids = model.generate(**inputs, max_new_tokens=20, bad_words_ids=bad_words_ids)
  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)

  raw_output[b] = []
  for answer in generated_text:
    raw_output[b].append(answer)

  b+= 1

  # Write file every 10 batches
  if b%10==0:
    print(f'Saving after batch {b}')
    with open(os.path.join(save_path, 'raw_outputs.json'), 'w+') as f:
      json.dump(raw_output, f)

  for i in range(len(images)):
    images[i].close()
  del images

  # break

with open(os.path.join(save_path, 'raw_outputs.json'), 'w+') as f:
    json.dump(raw_output, f)

Saving after batch 10
Saving after batch 20
Saving after batch 30
Saving after batch 40
Saving after batch 50
Saving after batch 60
Saving after batch 70
Saving after batch 80
Saving after batch 90
Saving after batch 100
Saving after batch 110
Saving after batch 120
Saving after batch 130
Saving after batch 140
Saving after batch 150
Saving after batch 160
Saving after batch 170
Saving after batch 180
Saving after batch 190
Saving after batch 200
Saving after batch 210
Saving after batch 220
Saving after batch 230
Saving after batch 240
Saving after batch 250
Saving after batch 260
Saving after batch 270
Saving after batch 280
Saving after batch 290
Saving after batch 300
Saving after batch 310
Saving after batch 320
Saving after batch 330
Saving after batch 340
Saving after batch 350
Saving after batch 360
Saving after batch 370
Saving after batch 380
Saving after batch 390
Saving after batch 400
Saving after batch 410
Saving after batch 420
Saving after batch 430
Saving after batch 4